In [1]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.device_count()) 
print(torch.cuda.get_device_name(0))  

True
1
NVIDIA GeForce RTX 2060


In [2]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

# 1. Model ve tokenizer'ı yükle
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [3]:
"""
# GPT-2'ye padding token ekle
tokenizer.pad_token = tokenizer.eos_token

# 2. Fine-tuning yapmak için bir veri seti yükle (Wikitext)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# 3. Veri setini eğitim ve değerlendirme (validation) olarak ikiye böl
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]

# 4. Veriyi GPT-2 için tokenle ve 'labels' ekle
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()  # 'labels' olarak 'input_ids' kullan
    return tokens

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

# 5. Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./results",                    # Çıktı dizini
    evaluation_strategy="epoch",               # Her epoch sonunda değerlendirme
    per_device_train_batch_size=2,             # Her cihazda batch boyutu
    per_device_eval_batch_size=2,              # Her cihazda evaluation batch boyutu
    num_train_epochs=3,                        # Eğitim epoch sayısı
    save_steps=10_000,                         # Model kaydetme adımı
    save_total_limit=2,                        # Maksimum 2 checkpoint sakla
    logging_dir='./logs',                      # Log dosyası dizini
    logging_steps=200,                         # Her 200 adımda bir log yaz
)

# 6. Eğitici tanımla
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,  # Değerlendirme veri seti ekleniyor
)

# 7. Modeli eğit
trainer.train()
"""

'\n# GPT-2\'ye padding token ekle\ntokenizer.pad_token = tokenizer.eos_token\n\n# 2. Fine-tuning yapmak için bir veri seti yükle (Wikitext)\ndataset = load_dataset("wikitext", "wikitext-2-raw-v1")\n\n# 3. Veri setini eğitim ve değerlendirme (validation) olarak ikiye böl\ntrain_dataset = dataset["train"]\nvalidation_dataset = dataset["validation"]\n\n# 4. Veriyi GPT-2 için tokenle ve \'labels\' ekle\ndef tokenize_function(examples):\n    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)\n    tokens["labels"] = tokens["input_ids"].copy()  # \'labels\' olarak \'input_ids\' kullan\n    return tokens\n\ntokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)\ntokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)\n\n# 5. Eğitim ayarları\ntraining_args = TrainingArguments(\n    output_dir="./results",                    # Çıktı dizini\n    evaluation_strategy="epoch",               # Her epoch sonu

In [4]:
# Modeli kaydetme
#trainer.save_model("./trained_model")  # Model ./trained_model klasörüne kaydedilecek

In [5]:
model = GPT2LMHeadModel.from_pretrained("./trained_model")

In [6]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def chat_with_model():
    print("Sohbet botuna hoş geldiniz! Su anda sadece ingilizce sohbet gecerlidir. (Çıkmak için 'exit' yazın)")
    
    while True:
        # Kullanıcıdan bir soru al
        user_input = input("Sen: ")
        
        # Çıkış için 'exit' komutunu kontrol et
        if user_input.lower() == "exit":
            print("Sohbet sona erdi.")
            break

        # Kullanıcı girdisini tokenleştir
        inputs = tokenizer(user_input, return_tensors="pt")
        
        # Model ile yanıt üret
        output = model.generate(
            inputs.input_ids, 
            max_length=100,  # Üretilecek maksimum token sayısı
            num_return_sequences=1,  # Kaç yanıt üretileceği
            no_repeat_ngram_size=2,  # Aynı cümlenin tekrarını önlemek için
            early_stopping=True
        )

        # Tokenleri metne dönüştür
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Modelin yanıtını göster
        print(f"Bot: {response}")

# Sohbet başlat
chat_with_model()


Sohbet botuna hoş geldiniz! Su anda sadece ingilizce sohbet gecerlidir. (Çıkmak için 'exit' yazın)


Sen:  Oh hello my lord


C:\Users\Ufuk\anaconda3\envs\environment2\lib\site-packages\transformers\generation\configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: Oh hello my lord , my sweetheart . I am your sweetest , the sweet one . 



Sen:  Are u a chatbot?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Bot: Are u a chatbot? " is a common question in chat rooms . 

